## 🌐 Desarrollo de funciones a consumir por la API



#### 📥Importaciones 

In [1]:

# Importación del módulo gc para la recolección de basura
import gc
# Importación del módulo pickle para la serialización de objetos
# Importación del módulo warnings para controlar advertencias
import warnings
# Importación de pandas y se asigna el alias 'pd' para su uso
import pandas as pd

# Importación de funciones desde data_utils


# Configuración de la omisión de advertencias
warnings.filterwarnings("ignore")

# Carga de la extensión autoreload y configuración para recargar automáticamente módulos
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

#### 📦 Extracción de los conjuntos de datos
* ✍️ **Extraemos df_reviews:** usuarios que realizaron reseñas de los juegos que consumen.
* 🎮 **Extraemos df_games:** juegos disponibles en la plataforma.
* 📊 **Extraemos df_items:** consumo de los juegos por parte de los usuarios.


In [6]:
df_steam_games =  pd.read_parquet('../0 Dataset/1.1_steam_games_LISTO.parquet')
df_user_items =   pd.read_parquet('../0 Dataset/1.2_user_items_LISTO.parquet')
df_user_reviews = pd.read_parquet('../0 Dataset/1.3_user_review_LISTO.parquet')


#### 1 🌐 `developer(desarrollador: str)`:

Devuelve la cantidad de juegos y porcentaje de contenido Free por año según empresa desarrolladora

🌐Para acelerar la velocidad de la api, he decidido crear un DF especifico. 

In [ ]:
df_API_developer = df_steam_games[["developer","user_id", "release_year", "price"]]
df_API_developer.to_parquet("../0 Dataset/df_API_developer.parquet")

In [ ]:
def developer(desarrollador: str):
    df_dev = df_API_developer[df_API_developer['developer'] == desarrollador]

    grouped = df_dev.groupby('release_year').agg(
        items=('user_id', 'count'),  # Cambiar de (x == 0).sum() a 'count' o 'sum'
        gratis=('price', lambda x: (x == 0).sum())
    )

    result = []
    for year, row in grouped.iterrows():
        juegos = int(row["items"])
        gratis_percent = round(row["gratis"] / juegos * 100, 2) if juegos > 0 else 0
        result.append({
            "Año": int(year),
            "Juegos": juegos,
            "Gratis %": gratis_percent
        })
    
    # Llamamos al recolector de basura
    gc.collect()

    return result
# Example usage
resultado = developer('Valve')
print(resultado)

[{'Año': 1998, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 1999, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2000, 'Juegos': 2, 'Gratis %': 0.0}, {'Año': 2001, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2003, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2004, 'Juegos': 5, 'Gratis %': 0.0}, {'Año': 2005, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2006, 'Juegos': 2, 'Gratis %': 0.0}, {'Año': 2007, 'Juegos': 3, 'Gratis %': 0.0}, {'Año': 2008, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2009, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2010, 'Juegos': 2, 'Gratis %': 50.0}, {'Año': 2011, 'Juegos': 1, 'Gratis %': 0.0}, {'Año': 2012, 'Juegos': 2, 'Gratis %': 0.0}, {'Año': 2014, 'Juegos': 7, 'Gratis %': 0.0}, {'Año': 2016, 'Juegos': 1, 'Gratis %': 100.0}, {'Año': 2017, 'Juegos': 2, 'Gratis %': 0.0}]


#### 🌐 `userdata(User_id: str)` : 
- Dinero gastado por el usuario, porcentaje de recomendación y cantidad de items.


In [ ]:
def userdata(user_id: str):
    # Filtrar las revisiones del usuario
    user_reviews = df_user_reviews[df_user_reviews['user_id'] == user_id]

    # Calcular el dinero gastado por el usuario
    spent_money = 0
    for item_id in user_reviews['reviews_item_id']:
        price = df_steam_games[df_steam_games['user_id'] == item_id]['price'].values
        if len(price) > 0:  # Verificar si se encontró el precio del ítem
            spent_money += price[0]

    # Calcular el porcentaje de recomendación
    total_reviews = len(user_reviews)
    num_recommendations = (user_reviews['sentiment_analysis'] == True).sum()
    recommendation_percentage = (num_recommendations / total_reviews) * 100 if total_reviews > 0 else 0

    # Crear el diccionario de salida
    result = {
        'Usuario': user_id,
        'Dinero gastado': f"{spent_money} USD",
        '% de recomendación': f"{recommendation_percentage}%",
        'cantidad de items': total_reviews
    }

    # Devolver el diccionario como parte de una lista
    return [result]

In [ ]:
# Ejemplo de uso
response = userdata("evcentric")
print(response)

[{'Usuario': 'evcentric', 'Dinero gastado': '54 USD', '% de recomendación': '40.0%', 'cantidad de items': 5}]


## Las funciones faltantes fueron guardadas en un borrador en local para su futuro desarrollo.

#### 🌐 `UserForGenre(genero)`:

- Usuario con más horas jugadas para el género dado y acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: "Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
Ranking de géneros por tiempo de juego